In [1]:
import pandas
import spacy
import unicodedata

In [2]:
nlp = spacy.load('en_core_web_sm')
#from spacy.lang.vi import Vietnamese
#nlp = Vietnamese()

In [3]:
df_raw = pandas.read_csv("batch2_vi_en_chat.csv", encoding = "utf-8")
df_raw.rename(columns = {'Original':'sep_msg', 
                    'Language Correction': 'clean_msg'}, inplace=True)
df_raw.head()

,Code,sep_msg,clean_msg,Translation,Note
0,2,【包邮led显微镜光源显微镜外加光源ty-60显微镜辅助光源灯架环形】https://m.t...,NaN,NaN,NOT VI/EN
1,3,ji tian dao yue nan,NaN,NaN,NOT VI/EN
2,4,old man head lapel pins multiple eyes edgar al...,NaN,NaN,NOT VI/EN
3,5,"oh, tai qui le",NaN,NaN,NOT VI/EN
4,6,kami mengudara di masa lalu,NaN,NaN,NOT VI/EN


In [4]:
df = df_raw[['sep_msg', 'clean_msg']]
df.dropna(inplace=True)
df = df.applymap(lambda x: x.replace('"', ''))
df.head()

/Users/yoong.thuydung/python-virtual-environments/mlt00/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sep_msg,clean_msg
220,đặt hàng cmm làm gì mà cả tháng ko đc,Đặt hàng *** nhưng sao cả tháng vẫn chưa có?
221,làm ăn gì nói như con kạc thuê mấy thèn khốn p...,làm ăn gì nói như **** thuê mấy thằng ***** ph...
222,mả tổ nhà mày đăg hình một đằg giao hàng một nẻo,"Bạn đăng hình một đằng, giao hàng một nẻo."
223,dm kg co hang thi huy don,*** không có hàng thì hủy đơn
224,vkl ma ko biet san pham ah,Mà bạn không biết tên sản phẩm là gì à?


In [5]:
list(df.columns.values)

['sep_msg', 'clean_msg']

In [6]:
#df.sep_msg = df.sep_msg.apply(lambda val: unicodedata.normalize('NFKD', str(val)).encode('ascii', 'ignore').decode())
#df.clean_msg = df.clean_msg.apply(lambda val: unicodedata.normalize('NFKD', str(val)).encode('ascii', 'ignore').decode())

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120621 entries, 220 to 121638
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sep_msg    120621 non-null  object
 1   clean_msg  120621 non-null  object
dtypes: object(2)
memory usage: 2.8+ MB


In [8]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace(r'\n',". ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<eos> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<eos> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<eos>")
    text = text.replace("?","?<eos>")
    text = text.replace("!","!<eos>")
    text = text.replace("<prd>",".")
    sentences = text.split("<eos>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [9]:
counter = 0
with open("correction1/processed_vn.txt", 'w') as f:
    for y, row in df.iterrows():
        orig = str(row.sep_msg).replace(r'\n'," ")
        clean = str(row.clean_msg).replace(r'\n'," ")
        #if clean == "nan":
            #orig = ""
        orig = orig.split("<eos>")
        clean = clean.split("<eos>")
        if not len(orig) == len(clean):
            counter +=1
            #print(orig)
            #print(clean)
            orig = split_into_sentences(row.sep_msg)
            clean = split_into_sentences(row.clean_msg)
        if not len(orig) == len(clean):
            continue
        for i in range(len(orig)):
            src = " ".join([str(x).strip() for x in nlp(orig[i].strip())])
            #print(src)
            trg = " ".join([str(x).strip() for x in nlp(clean[i].strip())])
            #print(clean)
            if len(src) < 3:
                continue
            if len(trg) < 3:
                continue
            f.write(src)
            f.write('\t')
            f.write(trg)
            f.write('\n')
            
print(counter)

FileNotFoundError: [Errno 2] No such file or directory: 'correction1/processed_vn.txt'

In [ ]:
#src_file = open("vn.src", "w")
#trg_file = open("vn.trg", "w")
with open('processed_vn.txt', 'r') as f:
    lines = [line for line in f]
    n = len(lines)
    train1 = lines[0:int(0.8*n)]
    train = lines[0:int(0.8*len(train1))]
    valid = lines[int(-0.2*len(train1)):]
    test = lines[int(-0.2*n):]
    
    for line in train:
        content = line.split('\t')
        src = content[0]
        trg = content[1]
        with open('correction1/train_vn.src', 'a') as src_file:
            src_file.write(src)
            src_file.write('\n')
        with open('correction1/train_vn.trg', 'a') as trg_file:
            trg_file.write(trg)
            #trg_file.write('\n')
            
    for line in test:
        content = line.split('\t')
        src = content[0]
        trg = content[1]
        with open('correction1/test_vn.src', 'a') as src_file:
            src_file.write(src)
            src_file.write('\n')
        with open('correction1/test_vn.trg', 'a') as trg_file:
            trg_file.write(trg)
            #trg_file.write('\n')
            
    for line in valid:
        content = line.split('\t')
        src = content[0]
        trg = content[1]
        with open('correction1/valid_vn.src', 'a') as src_file:
            src_file.write(src)
            src_file.write('\n')
        with open('correction1/valid_vn.trg', 'a') as trg_file:
            trg_file.write(trg)
            #trg_file.write('\n')
    
#     reader = csv.reader(f, dialect='excel', delimiter='\t')
#     for row in reader